### Reference:
Lesson 3: CNNs in TensorFlow  
Section 7. CNNs in TensorFlow (the content in this section refers to the code from [here](https://d17h27t6h515a5.cloudfront.net/topher/2017/February/58a61ca1_cnn/cnn.zip)

### CNN Architecture
The structure of this network is based on __LENET__ by _Yann Lecun_.  
The layers are (read from bottom to top)  
> Classifier  
> Fully Connected Layer _(out)_  
> Fully Connected Layer _(d1)_  
> Max Pooling Layer  
> Convolution Layer _(c2)_  
> Max Pooling Layer  
> Convolution Layer _(c1)_  
> Input  

### Dataset  
We are importing the MNIST dataset and using a convenient TensorFlow function to batch, scale, and one-hot encode the data.

In [1]:
# Download the dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

DEBUG = True
if DEBUG:
    x_train, y_train = mnist.train.images, mnist.train.labels
    x_val, y_val     = mnist.validation.images, mnist.validation.labels
    x_test, y_test   = mnist.test.images, mnist.test.labels
    print('Number of training samples: {}'.format(len(x_train)))
    print('Number of validation samples: {}'.format(len(x_val)))
    print('Number of test samples: {}'.format(len(x_test)))
    print('Shape of image: {}'.format(x_train[0].shape))
          

/home/supannee/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz
Number of training samples: 55000
Number of validation samples: 5000
Number of test samples: 10000
Shape of image: (28, 28, 1)


In [2]:
import tensorflow as tf

# parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# number of samples to calculate validation and accuracy 
# we can decrease this number as it depends on the memory size
test_valid_size = 256

# network parameters
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # probability to keep units


### Weights and Biases

We will create 3 layers alternating between convolutions and max pooling followed by a fully connected and output layer. We begin by defining the necessary weights and biases.

In [3]:
# weights
weights = {
    'wc1': tf.Variable(tf.random_normal([5,5,1,32])), # 32 5x5 conv filter
    'wc2': tf.Variable(tf.random_normal([5,5,32,64])), # 64 5x5 conv filter
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])), # fully connected with 1024 neurons
    'out': tf.Variable(tf.random_normal([1024, n_classes])) # fully connected with n_classes output
}

# biases
biases = {
    'bc1': tf.Variable(tf.random_normal([32])), # 32 neurons from conv1
    'bc2': tf.Variable(tf.random_normal([64])), # 64 neurons from conv2
    'bd1': tf.Variable(tf.random_normal([1024])), # 1024 neursons from fully connected layer 1
    'out': tf.Variable(tf.random_normal([n_classes])) # n_classes neurons
}

print('wc1 shape: {}'.format(weights['wc1'].shape))
print('wc2 shape: {}'.format(weights['wc2'].shape))
print('wd1 shape: {}'.format(weights['wd1'].shape))
print('out shape: {}'.format(weights['out'].shape))

wc1 shape: (5, 5, 1, 32)
wc2 shape: (5, 5, 32, 64)
wd1 shape: (3136, 1024)
out shape: (1024, 10)


### Convolutional Layers

TensorFlow provides the `tf.nn.conv2d()`, `tf.nn.bias_add()`, and `tf.nn.relu()` functions to create our own convolutional layers.  

To simplify the code, we define a convolution function below which does:  
>(1) compute the convolution between input `x` and weight `w`  
>(2) add bias  
>(3) apply a ReLU activation function

In [4]:
def conv2d(x, W, b, strides = 1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x)

__NB__ `'pading' = 'SAME'` means that padding is applied if needed to make the input valid when perform the convolution (or other operations).

### Max Pooling Layers

TensorFlow provides the tf.nn.max_pool() function to perform max pooling. To simplify the code, we define a function below:

In [5]:
# apply max pooling to x using a filter of size k
def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize = [1, k, k, 1],
        strides = [1, k, k, 1],
        padding = 'SAME')

### Model

This code construct the network to following LENET architecture.

In [6]:
def conv_net(x, weights, biases, dropout):
    # Convolutional Layer 1 - 28*28*1
    conv1 = conv2d(x, weights['wc1'], biases['bc1']) # out: (?x28x28x32)
    # Pooling Layer 1
    pool1 = maxpool2d(conv1, k=2) # out: (?x14x14x32)
    # Convolutional Layer 2
    conv2 = conv2d(pool1, weights['wc2'], biases['bc2']) # out: (?x14x14x64)
    # Pooling Layer 2
    pool2 = maxpool2d(conv2, k=2) # out: (?x7x7x64)
    # Fully Connected Layer 1
    # Reshape from Nx7x7x64 to 2D of size (? x 7*7*64)
    fc1 = tf.reshape(pool2, [-1, weights['wd1'].get_shape().as_list()[0]]) # out: (? x 7*7*64)
    #print('fc1 shape: {}'.format(fc1.shape))
    #                   (N x 7*7*64) x (7*7*64 x 1024)  
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1']) # out: (?x1024)
    fc1 = tf.nn.relu(fc1) # out: (?x1024)
    fc1 = tf.nn.dropout(fc1, dropout) # out: (?x1024)
    # Fully Connected Layer 2 (output)
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out']) # out: (?x10)
    return out # logits

In [7]:
# Construct the architecture

# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
# tf.reduce_mean() - compute the mean of elements across dimensions of a tensor
# This line is to compute the lost from the forward pass (through conv_net)
#        per batch
cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = y))
# Given the lost, run the optimiser to update the weights and biases
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(cost)

# Compute the accuracy 
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [8]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    # for each epoch
    for epoch in range(epochs): # 0,1,..,epochs
        # for each batch
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout})
            
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.0})
            
            # Calculate accuracy (validation)
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.0})
            
            print('Epoch {:>2}, Batch {:>3} - '
                 'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                     epoch+1, batch+1, loss, valid_acc))
            
    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))    

Epoch  1, Batch   1 - Loss: 64916.7539 Validation Accuracy: 0.132812
Epoch  1, Batch   2 - Loss: 46013.2852 Validation Accuracy: 0.121094
Epoch  1, Batch   3 - Loss: 46094.3594 Validation Accuracy: 0.121094
Epoch  1, Batch   4 - Loss: 40524.4336 Validation Accuracy: 0.113281
Epoch  1, Batch   5 - Loss: 34069.6914 Validation Accuracy: 0.148438
Epoch  1, Batch   6 - Loss: 30898.7773 Validation Accuracy: 0.140625
Epoch  1, Batch   7 - Loss: 30858.2773 Validation Accuracy: 0.125000
Epoch  1, Batch   8 - Loss: 31650.8516 Validation Accuracy: 0.097656
Epoch  1, Batch   9 - Loss: 31099.8887 Validation Accuracy: 0.105469
Epoch  1, Batch  10 - Loss: 25905.0820 Validation Accuracy: 0.125000
Epoch  1, Batch  11 - Loss: 22971.1465 Validation Accuracy: 0.152344
Epoch  1, Batch  12 - Loss: 21748.4414 Validation Accuracy: 0.156250
Epoch  1, Batch  13 - Loss: 23386.1875 Validation Accuracy: 0.152344
Epoch  1, Batch  14 - Loss: 22086.7070 Validation Accuracy: 0.160156
Epoch  1, Batch  15 - Loss: 21676.

Epoch  1, Batch 122 - Loss:  4633.1558 Validation Accuracy: 0.570312
Epoch  1, Batch 123 - Loss:  5078.6831 Validation Accuracy: 0.574219
Epoch  1, Batch 124 - Loss:  4623.6519 Validation Accuracy: 0.574219
Epoch  1, Batch 125 - Loss:  4296.1836 Validation Accuracy: 0.574219
Epoch  1, Batch 126 - Loss:  3751.2441 Validation Accuracy: 0.578125
Epoch  1, Batch 127 - Loss:  4814.8154 Validation Accuracy: 0.574219
Epoch  1, Batch 128 - Loss:  3871.8230 Validation Accuracy: 0.582031
Epoch  1, Batch 129 - Loss:  5241.3965 Validation Accuracy: 0.578125
Epoch  1, Batch 130 - Loss:  3413.0703 Validation Accuracy: 0.582031
Epoch  1, Batch 131 - Loss:  3848.8169 Validation Accuracy: 0.582031
Epoch  1, Batch 132 - Loss:  3927.1592 Validation Accuracy: 0.582031
Epoch  1, Batch 133 - Loss:  4496.6504 Validation Accuracy: 0.582031
Epoch  1, Batch 134 - Loss:  3755.9082 Validation Accuracy: 0.593750
Epoch  1, Batch 135 - Loss:  3598.1704 Validation Accuracy: 0.585938
Epoch  1, Batch 136 - Loss:  3927.

Epoch  1, Batch 241 - Loss:  3158.9956 Validation Accuracy: 0.664062
Epoch  1, Batch 242 - Loss:  2735.2935 Validation Accuracy: 0.671875
Epoch  1, Batch 243 - Loss:  2688.8760 Validation Accuracy: 0.667969
Epoch  1, Batch 244 - Loss:  2362.8765 Validation Accuracy: 0.675781
Epoch  1, Batch 245 - Loss:  2910.2434 Validation Accuracy: 0.664062
Epoch  1, Batch 246 - Loss:  2228.3745 Validation Accuracy: 0.667969
Epoch  1, Batch 247 - Loss:  2907.1731 Validation Accuracy: 0.664062
Epoch  1, Batch 248 - Loss:  1705.7058 Validation Accuracy: 0.671875
Epoch  1, Batch 249 - Loss:  3280.5227 Validation Accuracy: 0.675781
Epoch  1, Batch 250 - Loss:  2107.8008 Validation Accuracy: 0.683594
Epoch  1, Batch 251 - Loss:  2962.2510 Validation Accuracy: 0.671875
Epoch  1, Batch 252 - Loss:  2708.1455 Validation Accuracy: 0.671875
Epoch  1, Batch 253 - Loss:  2665.8662 Validation Accuracy: 0.683594
Epoch  1, Batch 254 - Loss:  2215.9319 Validation Accuracy: 0.675781
Epoch  1, Batch 255 - Loss:  2790.

Epoch  1, Batch 361 - Loss:  2249.5857 Validation Accuracy: 0.730469
Epoch  1, Batch 362 - Loss:  2026.9484 Validation Accuracy: 0.730469
Epoch  1, Batch 363 - Loss:  1828.7332 Validation Accuracy: 0.730469
Epoch  1, Batch 364 - Loss:  1528.6815 Validation Accuracy: 0.734375
Epoch  1, Batch 365 - Loss:  1109.4376 Validation Accuracy: 0.730469
Epoch  1, Batch 366 - Loss:  1626.4666 Validation Accuracy: 0.722656
Epoch  1, Batch 367 - Loss:  1342.4684 Validation Accuracy: 0.718750
Epoch  1, Batch 368 - Loss:  1953.3625 Validation Accuracy: 0.726562
Epoch  1, Batch 369 - Loss:  1370.2042 Validation Accuracy: 0.730469
Epoch  1, Batch 370 - Loss:  1200.4121 Validation Accuracy: 0.742188
Epoch  1, Batch 371 - Loss:  1573.6472 Validation Accuracy: 0.742188
Epoch  1, Batch 372 - Loss:  1399.7687 Validation Accuracy: 0.738281
Epoch  1, Batch 373 - Loss:  1288.3890 Validation Accuracy: 0.742188
Epoch  1, Batch 374 - Loss:  1363.0618 Validation Accuracy: 0.746094
Epoch  1, Batch 375 - Loss:  1499.

Epoch  2, Batch  57 - Loss:  1257.4807 Validation Accuracy: 0.773438
Epoch  2, Batch  58 - Loss:   969.3841 Validation Accuracy: 0.773438
Epoch  2, Batch  59 - Loss:  1443.9375 Validation Accuracy: 0.777344
Epoch  2, Batch  60 - Loss:  1453.7302 Validation Accuracy: 0.781250
Epoch  2, Batch  61 - Loss:  1160.5200 Validation Accuracy: 0.781250
Epoch  2, Batch  62 - Loss:  2041.7969 Validation Accuracy: 0.781250
Epoch  2, Batch  63 - Loss:  1358.1362 Validation Accuracy: 0.777344
Epoch  2, Batch  64 - Loss:  1375.0261 Validation Accuracy: 0.769531
Epoch  2, Batch  65 - Loss:  1112.4504 Validation Accuracy: 0.781250
Epoch  2, Batch  66 - Loss:  1409.9973 Validation Accuracy: 0.773438
Epoch  2, Batch  67 - Loss:  1555.6108 Validation Accuracy: 0.777344
Epoch  2, Batch  68 - Loss:  1521.8333 Validation Accuracy: 0.777344
Epoch  2, Batch  69 - Loss:  1297.7102 Validation Accuracy: 0.781250
Epoch  2, Batch  70 - Loss:   948.4194 Validation Accuracy: 0.777344
Epoch  2, Batch  71 - Loss:  1470.

...

Epoch  9, Batch 317 - Loss:   146.5425 Validation Accuracy: 0.781250
Epoch  9, Batch 318 - Loss:   330.7811 Validation Accuracy: 0.785156
Epoch  9, Batch 319 - Loss:   306.7155 Validation Accuracy: 0.781250
Epoch  9, Batch 320 - Loss:   219.6745 Validation Accuracy: 0.781250
Epoch  9, Batch 321 - Loss:   289.2109 Validation Accuracy: 0.789062
Epoch  9, Batch 322 - Loss:   211.3126 Validation Accuracy: 0.789062
Epoch  9, Batch 323 - Loss:   269.1754 Validation Accuracy: 0.785156
Epoch  9, Batch 324 - Loss:   123.3646 Validation Accuracy: 0.781250
Epoch  9, Batch 325 - Loss:   229.2872 Validation Accuracy: 0.781250
Epoch  9, Batch 326 - Loss:   292.6970 Validation Accuracy: 0.777344
Epoch  9, Batch 327 - Loss:   281.8555 Validation Accuracy: 0.781250
Epoch  9, Batch 328 - Loss:   213.9359 Validation Accuracy: 0.785156
Epoch  9, Batch 329 - Loss:   309.4272 Validation Accuracy: 0.789062
Epoch  9, Batch 330 - Loss:   189.3373 Validation Accuracy: 0.789062
Epoch  9, Batch 331 - Loss:   221.

Epoch 10, Batch   7 - Loss:   158.2525 Validation Accuracy: 0.792969
Epoch 10, Batch   8 - Loss:   278.5294 Validation Accuracy: 0.785156
Epoch 10, Batch   9 - Loss:   143.1709 Validation Accuracy: 0.789062
Epoch 10, Batch  10 - Loss:   174.2173 Validation Accuracy: 0.789062
Epoch 10, Batch  11 - Loss:   262.4262 Validation Accuracy: 0.792969
Epoch 10, Batch  12 - Loss:   183.5147 Validation Accuracy: 0.785156
Epoch 10, Batch  13 - Loss:   184.1605 Validation Accuracy: 0.777344
Epoch 10, Batch  14 - Loss:   257.4719 Validation Accuracy: 0.789062
Epoch 10, Batch  15 - Loss:   226.9767 Validation Accuracy: 0.792969
Epoch 10, Batch  16 - Loss:   232.6056 Validation Accuracy: 0.785156
Epoch 10, Batch  17 - Loss:   182.5403 Validation Accuracy: 0.789062
Epoch 10, Batch  18 - Loss:   202.7410 Validation Accuracy: 0.785156
Epoch 10, Batch  19 - Loss:   208.3481 Validation Accuracy: 0.785156
Epoch 10, Batch  20 - Loss:   245.5742 Validation Accuracy: 0.781250
Epoch 10, Batch  21 - Loss:   288.

Epoch 10, Batch 127 - Loss:   186.6009 Validation Accuracy: 0.789062
Epoch 10, Batch 128 - Loss:   295.7074 Validation Accuracy: 0.785156
Epoch 10, Batch 129 - Loss:   112.9618 Validation Accuracy: 0.777344
Epoch 10, Batch 130 - Loss:   256.1296 Validation Accuracy: 0.769531
Epoch 10, Batch 131 - Loss:   194.6731 Validation Accuracy: 0.773438
Epoch 10, Batch 132 - Loss:   249.3653 Validation Accuracy: 0.773438
Epoch 10, Batch 133 - Loss:   139.6859 Validation Accuracy: 0.781250
Epoch 10, Batch 134 - Loss:   135.7215 Validation Accuracy: 0.773438
Epoch 10, Batch 135 - Loss:   281.3405 Validation Accuracy: 0.769531
Epoch 10, Batch 136 - Loss:   218.4393 Validation Accuracy: 0.769531
Epoch 10, Batch 137 - Loss:   188.4659 Validation Accuracy: 0.769531
Epoch 10, Batch 138 - Loss:   208.5656 Validation Accuracy: 0.773438
Epoch 10, Batch 139 - Loss:   158.6320 Validation Accuracy: 0.781250
Epoch 10, Batch 140 - Loss:   216.0646 Validation Accuracy: 0.769531
Epoch 10, Batch 141 - Loss:   223.

Epoch 10, Batch 246 - Loss:   180.9298 Validation Accuracy: 0.781250
Epoch 10, Batch 247 - Loss:   157.8926 Validation Accuracy: 0.781250
Epoch 10, Batch 248 - Loss:   245.1791 Validation Accuracy: 0.785156
Epoch 10, Batch 249 - Loss:   181.6899 Validation Accuracy: 0.789062
Epoch 10, Batch 250 - Loss:   203.7965 Validation Accuracy: 0.789062
Epoch 10, Batch 251 - Loss:   282.9271 Validation Accuracy: 0.785156
Epoch 10, Batch 252 - Loss:   246.3446 Validation Accuracy: 0.789062
Epoch 10, Batch 253 - Loss:   296.0452 Validation Accuracy: 0.789062
Epoch 10, Batch 254 - Loss:   195.7758 Validation Accuracy: 0.789062
Epoch 10, Batch 255 - Loss:   204.4857 Validation Accuracy: 0.781250
Epoch 10, Batch 256 - Loss:   195.9572 Validation Accuracy: 0.769531
Epoch 10, Batch 257 - Loss:   210.8873 Validation Accuracy: 0.765625
Epoch 10, Batch 258 - Loss:   296.2980 Validation Accuracy: 0.769531
Epoch 10, Batch 259 - Loss:   138.6203 Validation Accuracy: 0.777344
Epoch 10, Batch 260 - Loss:   240.

Epoch 10, Batch 367 - Loss:   251.9870 Validation Accuracy: 0.792969
Epoch 10, Batch 368 - Loss:   285.4860 Validation Accuracy: 0.804688
Epoch 10, Batch 369 - Loss:   204.1074 Validation Accuracy: 0.796875
Epoch 10, Batch 370 - Loss:   291.9662 Validation Accuracy: 0.812500
Epoch 10, Batch 371 - Loss:   304.0167 Validation Accuracy: 0.816406
Epoch 10, Batch 372 - Loss:   232.9433 Validation Accuracy: 0.808594
Epoch 10, Batch 373 - Loss:   176.9756 Validation Accuracy: 0.816406
Epoch 10, Batch 374 - Loss:   240.5829 Validation Accuracy: 0.812500
Epoch 10, Batch 375 - Loss:   249.2624 Validation Accuracy: 0.804688
Epoch 10, Batch 376 - Loss:   176.0187 Validation Accuracy: 0.785156
Epoch 10, Batch 377 - Loss:   201.7958 Validation Accuracy: 0.792969
Epoch 10, Batch 378 - Loss:   294.9440 Validation Accuracy: 0.785156
Epoch 10, Batch 379 - Loss:   130.2527 Validation Accuracy: 0.789062
Epoch 10, Batch 380 - Loss:   243.3114 Validation Accuracy: 0.781250
Epoch 10, Batch 381 - Loss:   201.